In [38]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
daily_path =  "C:\\Users\\goodluck\\Desktop\\DB\\daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

In [39]:
# re-open
with open(DCBS_path, 'rb') as f:  
    DCBS = pickle.load(f)
# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)

In [25]:
def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if low_prc(code, dt, lm, fcd[1]):
                satisfy.append(code)
        
    else:
        for code in codes:
            if DCBS[dt][code][fcd[0]] <= fcd[2] and  DCBS[dt][code][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy


def takeSecond(elem):
    return elem[1]

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DTBS['A'][code][dt]['dp']):
                lm = calculate_time(dt, 'p', 20)
                prc = get_prcs(code, lm)
                lp = absolute_low(prc, 0)
                rk.append((code,  DCBS[dt][code]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan( DCBS[dt][code][idx]):
                rk.append((code,  DCBS[dt][code][idx]))
        else:
            if not np.isnan( DCBS[dt][code][idx]):
                if  DCBS[dt][code][idx] != 999:
                    rk.append((code,  DCBS[dt][code][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]
  

In [26]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
td


'2023-04-27'

In [27]:
select = list(DCBS[td].keys())

flt_cd = [['dp', 100, 130]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]
for fcd in flt_cd:
    select = select_condition(select, td, fcd)
    
    
# rk_cd = [['lp', 10, 'a']]
rk_cd = [['dl', 600, 'a']]
rk_cd = [['ytm', 20, 'd']]
rk_cd = [['dl', 30, 'a']]
codes = rank(select, td, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(codes)

30

In [28]:
for r in codes:
    print(r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], "收盘价：", DCBS[td][r]['dp'],  "转股溢价率：", DCBS[td][r]['cpr'], "到期收益率：", DCBS[td][r]['ytm'],"换手率：", DCBS[td][r]['trt'], '市值',  DCBS[td][r]['bl'], '双低', DCBS[td][r]['dl'])

123063.SZ 大禹转债 300021.SZ 大禹节水 收盘价： 126.1 转股溢价率： 17.86 到期收益率： -0.15 换手率： 4.21 市值 6.346 双低 143.95999999999998
113649.SH 丰山转债 603810.SH 丰山集团 收盘价： 124.874 转股溢价率： 21.44 到期收益率： -0.72 换手率： 3.06 市值 4.994 双低 146.314
118016.SH 京源转债 688096.SH 京源环保 收盘价： 117.0 转股溢价率： 30.94 到期收益率： 0.77 换手率： 2.07 市值 3.33 双低 147.94
123141.SZ 宏丰转债 300283.SZ 温州宏丰 收盘价： 121.333 转股溢价率： 27.98 到期收益率： -0.02 换手率： 2.46 市值 3.211 双低 149.313
113532.SH 海环转债 603817.SH 海峡环保 收盘价： 110.25 转股溢价率： 39.14 到期收益率： -0.21 换手率： 2.51 市值 4.585 双低 149.39
111004.SH 明新转债 605068.SH 明新旭腾 收盘价： 120.754 转股溢价率： 28.91 到期收益率： -0.03 换手率： 4.7 市值 6.729 双低 149.66400000000002
113651.SH 松霖转债 603992.SH 松霖科技 收盘价： 119.244 转股溢价率： 31.54 到期收益率： -0.35 换手率： 3.56 市值 6.1 双低 150.784
113628.SH 晨丰转债 603685.SH 晨丰科技 收盘价： 114.072 转股溢价率： 38.73 到期收益率： 1.45 换手率： 2.38 市值 4.15 双低 152.802
127081.SZ 中旗转债 001212.SZ 中旗新材 收盘价： 122.717 转股溢价率： 30.16 到期收益率： -0.91 换手率： 6.53 市值 5.4 双低 152.877
123175.SZ 百畅转债 300614.SZ 百川畅银 收盘价： 122.516 转股溢价率： 30.39 到期收益率： -0.64 换手率： 3.35 市值 4.2 双低 152.906
113665

In [41]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
select = list(DCBS[td].keys())
for code in select:
    DCBS[td][code]['cat1'] = DTBS['B'][code]['cat1']
    DCBS[td][code]['cat2'] = DTBS['B'][code]['cat2']
    
# save
f_save = open(DCBS_path, 'wb')
pickle.dump(DCBS, f_save)
f_save.close()


In [34]:
DTBS['127082.SZ']

KeyError: '127082.SZ'